#### Our algorithm

In our algorithm after doing gradient descent, Laplace's method for sampling, and Monte Carlo approximation of Bayesian marginalization, we need to compute the (posterior) prediction `mean` and `variance` using the following (code snippet from `10`), before we can plot our results

    for i, x_ in enumerate(v_samples_test):
        predictive_samples = []
        for theta_sample in theta_samples:
            predictive_mean = v_samples_test[i, :] @ theta_sample
            predictive_var = sigma_d**2
            individual_pred_dist = np.random.normal(predictive_mean, np.sqrt(predictive_var), num_samples_predictive_per_theta)
            predictive_samples.extend(individual_pred_dist)

        posterior_mean_y[i] = np.mean(predictive_samples)
        posterior_var_y[i] = np.var(predictive_samples)

    k_star_star = rbf_kernel(x_test, x_test, sigma_k)

    # Correction step
    posterior_var_y = posterior_var_y - np.diag(v_samples_test@v_samples_test.T) + np.diag(k_star_star)

    posterior_std_y = np.sqrt(posterior_var_y)

#### Correction for one but not for the other...


In this computation, we simply take the `posterior_mean_y` from monte carlo as it is, while having to make correction to the posterior variance `posterior_var_y`

So, `why` is this?

Recall in `notes_04`, we described how our virtual samples from eigendecomposition are only approximations of true $\phi(x)$ induced by RBF kernel. Therefore, from where we get the approximated $\phi(x)$ matters

* In our algorithm, the $\phi(x)$ for `training data` are obtained using $X_k$, which is computed based on training data only

* However, the $\phi(x)$ for `testing data` are obtained using $X_{t,k}$, which is computed based on training data and testing data. As a result, the $\phi(x)$ for testing data can only preserve the correlation `between training and testing data`, it cannot preserve the correlation among testing data (which is defined by `k_star_star` in the code snippet)


Based on above, we can see that

* if the prediction requires anything related to correlation among testing data, then our approximation will be off and a correction is needed
* if the prediction does not require anything related to correlation among testing data, then our approximation will be fine

#### `Analytical solution` of posterior mean and covariance

We can see it through analytical solution of our algorithm (from eq 2.12 in Rasmussen and Williams book on Gaussian processes)

The prediction mean `posterior_mean_y` is computed as

$$X_\text{t,k}(X_k+\sigma_d^2I)^{-1}y$$

while the prediction covariance matrix (the diagonal of which is `posterior_var_y`) is computed as

$$X_\text{test_test}-X_\text{t,k}(X_k+\sigma_d^2I)^{-1}X_\text{t,k}^T$$

From these two equations, we can see that

* prediction `mean` only relies on $X_{t,k}$ and $X_k$, which can be well approximated using dot products of virtual samples obtained in our algorithm since these two are used to generate our virtual samples in the first place
* prediction `covariance` matrix requires computation of $X_{test,test}$, which our virtual samples cannot approximate well with dot products

As a result, only the prediction variance needs to be corrected, by

* first, subtracting the inaccurate estimation of `k_star_star` based on the dot product of virtual samples `- np.diag(self.v_samples_test @ self.v_samples_test.T)`
* then, adding back the true `k_star_star` based on RBF kernel function `+ np.diag(self.k_star_star)`

Since we only need diagonal components of covariance matrix for our purpose, which is to visualize the prediction variance at each `x_test`, in the future, we can avoid computation of full `k_star_star` and only compute the diagonal of this matrix (which includes just a vector of one's)